In [1]:
import math
import matplotlib as mpl
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation

plt.switch_backend('Qt5Agg')

In [22]:
def thehill(t):
    
    if t > .6 and t <= 2:
        return np.sin(2*np.pi*t)*3 + 1
    elif t > 2 and t <= 3:
        return 1.3
    elif t > 3 and t <= 4:
        return np.cos(2*np.pi*t) * 0.7 + 1
    elif t > 4 and t <= 6:
        return 6**-(t-4)
    elif t > 6 and t <= 8:
        return 1 - 6**-(t-6)
    elif t > 8 and t <= 10:
        return np.sin(2*np.pi*t) * 0.5 + 1 
    else:
        return 1.


In [23]:

fig = plt.figure(figsize=(20, 4))
ax1 = fig.add_subplot(111)

x = np.arange(0.0, 10.0, 0.01)
y = [thehill(i) for i in x]

ax1.plot(x, y)
plt.show()

In [4]:
class Terrain(object):
    
    def __init__(self, fn):
        self.fn = fn
               
    def get_y(self, x):
        return self.fn(x)

In [5]:
class TerrainProblem(object):
    def __init__(self, terrain, initial_x):
        self.terrain = terrain
        self.initial_x = initial_x
        self.step = 0.01
        self.min_x = 0.0
        self.max_x = 10.0

    
    def actions(self, state = None):
        return np.arange(self.min_x, self.max_x, self.step)
    
    
    def results(self, state, action):
        if action in self.actions():
            return action
        else:
            return None
        
        
    def goal_test(self, state):
        maxval = -float('inf')
        for i in self.actions():
            y = self.terrain.get_y(i)
            if y > maxval:
                maxval = y
                
        if maxval == state:
            return True
        else:
            return False
    
    
    def path_cost(self, c, state1, state2, action):
        return self.terrain.get_y(state2) - self.terrain.get_y(state1)
    

In [6]:
class Node(object):
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __lt__(self, node):
        return self.y < node.y
    
    def __repr__(self):
        return "({0}, {1})".format(self.x, self.y)
    
    def __eq__(self, other):
        return isinstance(other, Node) and self.x == other.x and self.y == self.y    
    
    def __hash__(self):
        return hash((self.x, self.y))


In [7]:

def hill_climb(tp):
    current = Node(tp.initial_x, tp.terrain.fn(tp.initial_x))
    path = []
    
    safety = 500
    
    while True:
        
        safety -= 1
        if safety <= 0:
            break
        
        nl = Node(current.x - tp.step, tp.terrain.fn(current.x - tp.step))
        nr = Node(current.x + tp.step, tp.terrain.fn(current.x + tp.step))
        
        if nl.y == nr.y:
            n = nl
        else:
            n = max(nl, nr)
        
        path.append((current.x, current.y))
        
        if current.y >= n.y:
            return current, path
        else:
            current = n
            

In [8]:
t = Terrain(thehill)
tp = TerrainProblem(t,  6.1)
node, path = hill_climb(tp)

In [9]:

fig = plt.figure(figsize=(20, 4))
ax1 = fig.add_subplot(111)

x = tp.actions()
y = [tp.terrain.get_y(i) for i in x]
ax1.plot(x, y)


def animate(i, path):
    x, y = path[i]
    s = ax1.scatter(x, y, color='red', marker='x', s=10)
    return s


anim = animation.FuncAnimation(fig, animate, frames=len(path), interval=10, blit=False, fargs=[path])


plt.tight_layout()
plt.show()

In [4]:
import random
import sys
import math


def exp_schedule(k=20, lam=0.005, limit=1000):
    """One possible schedule function for simulated annealing"""
    return lambda t: (k * math.exp(-lam * t) if t < limit else 0)

def probability(p):
    """Return true with probability p."""
    return p > random.uniform(0.0, 1.0)

In [5]:
s = exp_schedule()

for t in range(1000):
    print(t, s(t))

0 20.0
1 19.900249583853647
2 19.800996674983363
3 19.70223879206125
4 19.603973466135105
5 19.506198240566654
6 19.40891067097016
7 19.31210832515133
8 19.215788783046463
9 19.119949636662
10 19.02458849001428
11 18.929702959069676
12 18.835290671684973
13 18.741349267548067
14 18.647876398118967
15 18.554869726571056
16 18.462326927732715
17 18.370245688029147
18 18.278623705424565
19 18.18745868936463
20 18.09674836071919
21 18.006490451725313
22 17.916682705930565
23 17.827322878136627
24 17.73840873434315
25 17.649938051691908
26 17.561908618411227
27 17.474318233760687
28 17.38716470797612
29 17.300445862214826
30 17.214159528501156
31 17.12830354967227
32 17.042875779324227
33 16.957874081758316
34 16.873296331927673
35 16.789140415384146
36 16.70540422822544
37 16.622085677042513
38 16.539182678867245
39 16.45669316112037
40 16.374615061559638
41 16.29294632822829
42 16.21168491940374
43 16.13082880354654
44 16.05037595924957
45 15.970324375187541
46 15.89067205006668
47 15.811

In [63]:


def simulated_annealing(tp, schedule=exp_schedule(limit=3000)):
    current = Node(tp.initial_x, tp.terrain.fn(tp.initial_x))
    path = []
    
    for t in range(sys.maxsize):
        T = schedule(t)
        
        path.append((current.x, current.y))
        if T == 0:
            return current, path
        
        neighbors = tp.actions()
        
        next_x = random.choice(neighbors)
        next = Node(next_x, tp.terrain.fn(next_x))
        
        delta_e = next.y - current.y
        
        if delta_e > 0 or probability(math.exp(delta_e / T)):
            current = next

            

In [64]:
t = Terrain(thehill)
tp = TerrainProblem(t,  1)
node, path = simulated_annealing(tp)



In [66]:
path[:500]

[(1, 0.99999999999999922),
 (0.85999999999999999, -1.3115397283273689),
 (2.5800000000000001, 1.3),
 (4.2400000000000002, 0.6504946063462167),
 (2.6200000000000001, 1.3),
 (7.4299999999999997, 0.92286638035533552),
 (3.3799999999999999, 0.48972196080501207),
 (5.5200000000000005, 0.065646276766271619),
 (7.21, 0.88559711224780902),
 (9.3200000000000003, 1.4524135262330096),
 (0.53000000000000003, 1.0),
 (2.6400000000000001, 1.3),
 (9.9800000000000004, 0.93733338321784698),
 (9.5400000000000009, 0.87565505641756924),
 (4.6900000000000004, 0.29045284664940768),
 (4.6900000000000004, 0.29045284664940768),
 (9.9600000000000009, 0.87565505641757557),
 (3.5600000000000001, 0.34915645987822419),
 (4.9199999999999999, 0.19235383513061768),
 (7.8700000000000001, 0.96493634683702556),
 (2.9199999999999999, 1.3),
 (8.2599999999999998, 1.4990133642141359),
 (5.75, 0.043474571668702423),
 (6.71, 0.7197712999062329),
 (4.1299999999999999, 0.79221003152945402),
 (0.71999999999999997, -1.9468617521860

In [70]:

fig = plt.figure(figsize=(15, 10))
ax1 = fig.add_subplot(111)

x1 = tp.actions()
y1 = [tp.terrain.get_y(i) for i in x]
ax1.plot(x1, y1)


def animate(i, path):
    ax1.clear()
    ax1.set_ylim([-3, 6])
    ax1.plot(x1, y1)
    x, y = path[i]
    
    if i+1 == len(path) or y == 4:
        c = 'green'
    else:
        c = 'red'
    
    ax1.text(4, 4, str(i))
    l = ax1.plot([x, x], [y, -2], color=c)
    s = ax1.scatter(x, y, color=c, marker='o', s=10)
    
    return s


anim = animation.FuncAnimation(fig, animate, frames=len(path), interval=1, blit=False, fargs=[path])


plt.tight_layout(pad=10)
plt.show()